In [1]:
import os.path, sys

import numpy as np
import pandas as pd
from scipy import linalg
from scipy.sparse import csr_matrix
from sklearn.preprocessing import OneHotEncoder

import Bio.PDB, warnings
pdb_list = Bio.PDB.PDBList()
pdb_parser = Bio.PDB.PDBParser()
from scipy.spatial import distance_matrix
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)

from joblib import Parallel, delayed
import timeit

import matplotlib.pyplot as plt

# # --- Import our Code ---# #
#import emachine as EM
from direct_info import direct_info

# import data processing and general DCA_ER tools
from data_processing import data_processing_msa2pdb
import ecc_tools as tools
from pathlib import Path
np.random.seed(1)

from Bio import SeqIO
from Bio.PDB import *
from scipy.spatial import distance_matrix
from Bio import pairwise2
#from Bio.SubsMat.MatrixInfo import blosum62
pdb_parser = Bio.PDB.PDBParser()

from prody import *



In [7]:
# # load and convert pfam to numpy array.
msa_file = '/home/ecresswell/Pfam-A.full/PF00118_full.txt'

from ecc_tools import read_FASTA
msa_fasta, msa = read_FASTA(msa_file)
np.save('/home/ecresswell/Pfam-A.full/PF00118/msa.npy', msa)



Loading Fasta File
/home/ecresswell/Pfam-A.full/PF00118_full.txt



Exception ignored in: <_io.FileIO name='/home/ecresswell/Pfam-A.full/PF00118_full.txt' mode='rb' closefd=True>
Traceback (most recent call last):
  File "/tmp/ipykernel_107068/1521358331.py", line 5, in <module>


In [8]:
create_new = True
printing = True
removing_cols = True


data_path = Path('/data/cresswellclayec/DCA_ER/Pfam-A.full')
data_path = Path('/data/cresswellclayec/Pfam-A.full')
data_path = Path('/home/ecresswell/Pfam-A.full')

# Define data directories
DCA_ER_dir = '/data/cresswellclayec/DCA_ER' # Set DCA_ER directory
DCA_ER_dir = '/home/ecresswell/DCA_ER' # Set DCA_ER directory

biowulf_dir = '%s/biowulf_full' % DCA_ER_dir

out_dir = '%s/protein_data/di/' % biowulf_dir
processed_data_dir = "%s/protein_data/data_processing_output" % biowulf_dir
pdb_dir = '%s/protein_data/pdb_data/' % biowulf_dir



pfam_dir = "/fdb/fastadb/pfam"

from data_processing import pdb2msa, data_processing_pdb2msa


import gzip, shutil
def gunzip(file_path, output_path):
    print('Unzipping %s to %s' % (file_path, output_path))
    with gzip.open(file_path,"rb") as f_in, open(output_path,"wb") as f_out:
        shutil.copyfileobj(f_in, f_out)




n_cpus = 6
print('\n\nUnzipping %s' % pdb_path)





Unzipping /home/ecresswell/PDB/pdb1ass.ent.gz


In [9]:
# pdb_path = "/pdb/pdb/zd/pdb1zdr.ent.gz"
pdb_path = "/home/ecresswell/PDB/pdb1ass.ent.gz"
unzipped_pdb_filename = os.path.basename(pdb_path).replace(".gz", "")

pdb_out_path = "%s%s" % (pdb_dir, unzipped_pdb_filename)
print('Unzipping %s to %s' % (pdb_path, pdb_out_path))

gunzip(pdb_path, pdb_out_path)
print(pdb_out_path)
print(pdb_dir)
pdb2msa_results_1 = pdb2msa(pdb_out_path, pdb_dir, create_new=False)
print(pdb2msa_results)


if len(pdb2msa_results) > 1:
    fasta_file = pdb2msa_results_1[0]
    prody_df = pdb2msa_results_1[1]
else:
    prody_df_1 = pdb2msa_results_1[0]


print('\nPDB DF with associated Protein Families\n', prody_df.loc[:,  [column for column in prody_df.columns if column not in ['locations', 'PDB Sequence']]].head())
print("\n\nLooping through Prody Search DataFrame:", prody_df.head())
rows_to_drop = []
for ir, pdb2msa_row in enumerate(prody_df_1.iterrows()):
    print('\n\nGetting msa with following pdb2msa entry:\n', pdb2msa_row)
    #try:
    dp_result =  data_processing_pdb2msa(data_path, prody_df_1.iloc[pdb2msa_row[0]], gap_seqs=0.2, gap_cols=0.2, prob_low=0.004,
                               conserved_cols=0.8, printing=True, out_dir=processed_data_dir, pdb_dir=pdb_dir, letter_format=False,
                               remove_cols=True, create_new=True, n_cpu=min(2, n_cpus))
    if dp_result is not None:
        [s0, removed_cols, s_index, tpdb, pdb_s_index] = dp_result
        break
    else:
        rows_to_drop.append(ir)
        continue
    #except Exception as e:
    #    print('row %d got exception: ' % ir , e)
    #    print('moving on.. ')
    #    pass


pdb_id_1 = pdb2msa_row[1]['PDB ID']
pfam_id_1 = pdb2msa_row[1]['Pfam']
# update Prody search DF (use same filename as pdb2msa() in data_processing
prody_df_1 = prody_df.drop(rows_to_drop)
print("\nSaving updated Prody Search DataFrame:", prody_df_1.head())
prody_df_1.to_csv('%s/%s_pdb_df.csv' % (pdb_dir, pdb_id_1))

if dp_result is None:
    print('None of the available prody pdb search found matching alignments... Exiting..')
    sys.exit()
print('Done Preprocessing Data.....')


Unzipping /home/ecresswell/PDB/pdb1ass.ent.gz to /home/ecresswell/DCA_ER/biowulf_full/protein_data/pdb_data/pdb1ass.ent
Unzipping /home/ecresswell/PDB/pdb1ass.ent.gz to /home/ecresswell/DCA_ER/biowulf_full/protein_data/pdb_data/pdb1ass.ent
/home/ecresswell/DCA_ER/biowulf_full/protein_data/pdb_data/pdb1ass.ent
/home/ecresswell/DCA_ER/biowulf_full/protein_data/pdb_data/
[   Unnamed: 0 PDB ID Chain  Polypeptide Index     Pfam   accession   class  \
0           0   1a6e     A                  0  PF00118  PF00118.27  Domain   
1           1   1a6e     B                  0  PF00118  PF00118.27  Domain   

           id    type                                       PDB Sequence  \
0  Cpn60_TCP1  Pfam-A  REQGKNAQRNNIEAAKAIADAVRTTLGPKGMDKMLVDSIGDIIISN...   
1  Cpn60_TCP1  Pfam-A  KDAMKENIEAAIAISNSVRSSLGPRGMDKMLVDSLGDIVITNDGVT...   

   ali_end  ali_start  bitscore  end    cond_evalue     ind_evalue  \
0      503         18    615.03  503  4.800000e-189  9.500000e-185   
1      501         14   

-SGIVIDKEK-------VHS--KMPDVVKNAKIAL-IDSA-LEIKKTEI--EAKV--QISDPSKI---QDF------L---N-QETNTFKQ-MVEK-I--K-KSGANVVLCQK---GID--D-VAQ-HYLA---K--EGIYAVR-RV-K--KS-----D--M--EK------L---A--KAT---GAK--IVTD---LDD-LTPSVLGEAET--VEER--KIGD--------DRMTFVMGCK--
 |       |       |    |  |        | | |  |         || |  |   | ||   |        |   | |    |   | |  |  | |      |  |   |    | |   |      |  |   |   |  |  |      |  |  ||      |   |  |     | |  |      |   |    |       | |   ||          |     ||    
AS-------KAGLQTGRV--LLK--D--------LEI-S-GL-------QNEA-VHAQ---P-KIKRKQ--KIQYKSLAHPNIQ----F--AM-E-VIAAKRK------L--KHCYG--VNDKV--SH---NGVKSSE---A--KR-DKAAK-VVFGADAFMKSEKVISKQCLCYRAFLK--LEKG-KGLI---QFEL--GL----L-----MIV-E-QMKI--PKKHLALSD-----MG--LV
  Score=58

match upgrade at 87
87: pairwise score=58.000000

lengths:  151 151
Alignment(seqA='-SGIVIDKEK-------VHS--KMPDVVKNAKIAL-IDSA-LEIKKTEI--EAKV--QISDPSKI---QDF------L---N-QETNTFKQ-MVEK-I--K-KSGANVVLCQK---GID--D-VAQ-HYLA---K--EGIYAVR-RV-K--KS-----D--M--EK---

In [10]:
unzipped_pdb_filename = os.path.basename(pdb_path).replace(".gz", "")

pdb_out_path = "%s%s" % (pdb_dir, unzipped_pdb_filename)
print('Unzipping %s to %s' % (pdb_path, pdb_out_path))

gunzip(pdb_path, pdb_out_path)
print(pdb_out_path)
print(pdb_dir)
pdb2msa_results_2 = pdb2msa(pdb_out_path, pdb_dir, create_new=False)
print(pdb2msa_results)


if len(pdb2msa_results) > 1:
    fasta_file = pdb2msa_results_2[0]
    prody_df = pdb2msa_results_2[1]
else:
    prody_df_2 = pdb2msa_results_2[0]


print('\nPDB DF with associated Protein Families\n', prody_df_2.loc[:,  [column for column in prody_df_2.columns if column not in ['locations', 'PDB Sequence']]].head())
print("\n\nLooping through Prody Search DataFrame:", prody_df_2.head())
rows_to_drop = []
for ir, pdb2msa_row in enumerate(prody_df_2.iterrows()):
    print('\n\nGetting msa with following pdb2msa entry:\n', pdb2msa_row)
    #try:
    dp_result =  data_processing_pdb2msa(data_path, prody_df_1.iloc[pdb2msa_row[0]], gap_seqs=0.2, gap_cols=0.2, prob_low=0.004,
                               conserved_cols=0.8, printing=True, out_dir=processed_data_dir, pdb_dir=pdb_dir, letter_format=False,
                               remove_cols=True, create_new=True, n_cpu=min(2, n_cpus))
    if dp_result is not None:
        [s0, removed_cols, s_index, tpdb, pdb_s_index] = dp_result
        break
    else:
        rows_to_drop.append(ir)
        continue
    #except Exception as e:
    #    print('row %d got exception: ' % ir , e)
    #    print('moving on.. ')
    #    pass


pdb_id_2 = pdb2msa_row[1]['PDB ID']
pfam_id_2 = pdb2msa_row[1]['Pfam']
# update Prody search DF (use same filename as pdb2msa() in data_processing
prody_df_2 = prody_df.drop(rows_to_drop)
print("\nSaving updated Prody Search DataFrame:", prody_df_2.head())
prody_df_2.to_csv('%s/%s_pdb_df.csv' % (pdb_dir, pdb_id_2))

if dp_result is None:
    print('None of the available prody pdb search found matching alignments... Exiting..')
    sys.exit()
print('Done Preprocessing Data.....')

Unzipping /home/ecresswell/PDB/pdb1ass.ent.gz to /home/ecresswell/DCA_ER/biowulf_full/protein_data/pdb_data/pdb1ass.ent
Unzipping /home/ecresswell/PDB/pdb1ass.ent.gz to /home/ecresswell/DCA_ER/biowulf_full/protein_data/pdb_data/pdb1ass.ent
/home/ecresswell/DCA_ER/biowulf_full/protein_data/pdb_data/pdb1ass.ent
/home/ecresswell/DCA_ER/biowulf_full/protein_data/pdb_data/
[   Unnamed: 0 PDB ID Chain  Polypeptide Index     Pfam   accession   class  \
0           0   1a6e     A                  0  PF00118  PF00118.27  Domain   
1           1   1a6e     B                  0  PF00118  PF00118.27  Domain   

           id    type                                       PDB Sequence  \
0  Cpn60_TCP1  Pfam-A  REQGKNAQRNNIEAAKAIADAVRTTLGPKGMDKMLVDSIGDIIISN...   
1  Cpn60_TCP1  Pfam-A  KDAMKENIEAAIAISNSVRSSLGPRGMDKMLVDSLGDIVITNDGVT...   

   ali_end  ali_start  bitscore  end    cond_evalue     ind_evalue  \
0      503         18    615.03  503  4.800000e-189  9.500000e-185   
1      501         14   

IADAVRTTLGPKGMDKMLVDSIGDIIISNDGATILKEMDVEHPTAKMIVEVSKAQDTAVGDGTTTAVVLSGELLKQAETLLDQGVHPTVISNGYRLAVNEARKIIDEIAEKSTDDATLRKIALTALSGKNTGLSNDFLADLVVKAVNAVAEVRDGKTIVDTANIKVDKKNGGSVNDTQFISGIVIDKEKVHSKMPDVVKNAKIALIDSALEIKKTEIEAKVQISDPSKIQDFLNQETNTFKQMVEKIKKSGANVVLCQKGIDDVAQHYLAKEGIYAVRRVKKSDMEKLAKATGAKIVTDLDDLTPSVLGEAETVEERKIGDDRMTFVMGCKNPKAVSILIRGGTDHVVSEVERALNDAIRVVAITKEDGKFLWGGGAVEAELAMRLAKYANSVGGREQLAIEAFAKALEIIPRTLAENAGIDPINTLIKLKADDEKGRISVGVDLDNNGVGDMKAKGVVDPLRVKTHALESAVEVATMILRIDDVI
                                                                                                                                                                                                                      |              |  |                  |   |         |                                                                                                                       |           |                                                                                        
--------------------------

IADAVRTTLGPKGMDKMLVDSIGDIIISNDGATILKEMDVEHPTAKMIVEVSKAQDTAVGDGTTTAVVLSGELLKQAETLLDQGVHPTVISNGYRLAVNEARKIIDEIAEKSTDDATLRKIALTALSGKNTGLSNDFLADLVVKAVNAVAEVRDGKTIVDTANIKVDKKNGGSVNDTQFISGIVIDKEKVHSKMPDVVKNAKIALIDSALEIKKTEIEAKVQISDPSKIQDFLNQETNTFKQMVEKIKKSGANVVLCQKGIDDVAQHYLAKEGIYAVRRVKKSDMEKLAKATGAKIVTDLDDLTPSVLGEAETVEERKIGDDRMTFVMGCKNPKAVSILIRGGTDHVVSEVERALNDAIRVVAITKEDGKFLWGGGAVEAELAMRLAKYANSVGGREQLAIEAFAKALEIIPRTLAENAGIDPINTLIKLKADDEKGRISVGVDLDNNGVGDMKAKGVVDPLRVKTHALESAVEVATMILRIDDVI
                                                                                                                                     |                           | |          |                        |              |          |                          |                                                  | |                         |                       |        |                                |                                 |               |                           |    |     
--------------------------

IADAVRTTLGPKGMDKMLVDSIGDIII-SNDGATILKEMDVEHPTAKMIVEVSKAQDTAVGDGTTTAVVL-SGELLKQAETLLDQGVHPTVISNGYRLAVNEARKIIDEIAEKSTDDATLRKIALTALSGKNTGLSNDFLADLVVKAVNAVAEVRDGKTIVDTANIKVDKKNGGSVNDTQFISGIVIDKEKVHSKMPDVVKNAKIALIDSALEIKKTEIEAKVQISDPSKIQDFLNQETNTFKQMVEK-IKKSGANVV-LCQKGIDDVAQHYLAKEGIYAVRRVKKSDMEKLAKATGAKIVTDLDDLTPSVLGEAETVEERKIGDDRMTFVMGCKNPKAVSILIRGG-TDHVVSEVERALNDAIRVVAITKEDGKFLWGGGAVEAELAMRLAKYANSVGGREQLAIEAFAKALEIIPRTLAENAGIDPINTLIKLKADDEKGRISVGVDLDNNGVGDMKAKGVVDPLRVKTHALESAVEVATMILRIDDVI
     |                      |                   |                      |             |      |                  | |       ||                             |                                    |           |                ||        |                 |  |  |      |         |                |            |        |                        |  |  | ||     |    |                                          |                                               |                     |      |          |      
-----R----------

IADAVRTTLGPKGMDKMLVDSIGDIIISNDGATILKEMDVEHPTAKMIVEVSKAQDTAVGDGTTTAVVLSGELLKQAETLLDQGVHPTVISNGYRLAVNEA-RKIIDEIAEKSTDDATLRKIALT-ALSGKNTGLSNDFLADLVVKAVNAVAEVRDGKTIVDTANIKVDKKNGGSVNDTQFISGIVIDKEKVHSKMPDVVKNAKIALIDSALEIKKTEIEAKVQISDPSKIQDFLNQETNTFKQMVEKIKKSGANVVLCQKGIDDVAQHYLAKEGIYAVRRVKKSDM--EKLAKATGAKIVTDLDDLTPSVLGEAETVEERKIGDDRMTFVMGCKNPKAVSILIRGGTDHVVSE--VERALN---DAIRVVAI-TKEDGKFLWGGGAVEAELAMRL-AKYAN-SVGGREQ-LAIEAFAKALEIIPRTLA-ENAGIDPINTLIKLKADDEKGRI-SVGVDLDNNGVG--DMKAK-GVVDPLRVKTHALESAV-EVATMILRID---DVI--
       |      ||           |                  |                      |    |     |      |              |               |   |   |      |     | |        | |        | |                   |           |                       |      ||                                          |   |     |     |  |   |   |      ||     |       |                | |             |   |  ||    |        ||   |         |  |    ||    |       | |       |        |    |       |       |  |   ||  |     |   | |  | |      || 

lengths:  486 262
Alignment(seqA='IAD-AVRTTLGPKGMD-KMLVDSIGDII-ISNDGATIL---KEMDVEHPTA--K-MIVEV--SK----AQDTAVGDGTTTAV-----VL-SGELLK-QAETLLDQ--GV--HP--T---VI---S-----NGYRLAV-NEARKIIDEIAEK---STDDATLRKIALTALS-G-KNTGLSN-DF-L-AD------LVVKAVNAV----AEVRDGK--T-IVDT-AN-----IKVDKKN----GGSVNDTQFISGIVIDKEKVHSK-MPDV--VKNAKIALIDSALEIKKTEIEAKVQISDPSKIQDF-LNQETNTFKQ-MVE-KIKKSGANVVLCQKGIDDV-AQHY-LAKEGIYA-V--RRVKKSDMEKLAKATGAKIVTDLD-DLTPSVLGEAE--TVEERKIGDDRMTFV--MGCKNPK-AVS-ILIRGGTD--HVVSEVE--RAL--N-DAIRVVAITKEDGKFLWGG-GAVEAELAMRL-A-----KY---ANSVGGRE-Q-----LAIEAFAKALEIIPRTLAENAGIDPINTLI--K-LKADDEKGRI-SVGVDLDNNGVGDMKAKGVVDPL-RVKTHALESAVEVA---TM-I--LRID-DVI---', seqB='---SA-----------SKM-----G---HI-N--A---SNMKE---E----SIKLM---VHLS-HLGSA---A---G-----RSNHNV-NS-----HQ--T----KTG-NKH-LGTANLV-KGCSTSAGDNG----VIN---K---E--E-GMIS-----L---A--A--TGFK--G---QD-GLGADSGSSTGL--------CCGCA--R---SSTEIV-TEA-EHRGSI------ARLEGG--N-----SG--I--------LM---QAV-N-K-------L----T-----------------YL---------DM--RK-KK---------K-----TA-

IADAVRTTLGPKGMDKMLVD-----SIGDIIISND---GATI-LKEMDVEHP-TAK-MIVEV---SK---AQDTAVGDGTTTAVV-LSGE-LLKQA-E-TLLDQGV-HPTVISNGYRLAVNEA-RKI-IDE--IAEK---STD-DATLRKIAL-TALSGKNTGLSNDFLA-DLVVKAVNAVAEVRDGKTIVDTA--NIKVDK-KNGGSVND--TQFISGIVI---DK--E---K-VHSKMPDVV----------KN-----AK--IALI------D--S--ALE-----IKKT-E---I--------E--A--KV-QISDP----SKI-QD-FLN---QE-T-N-TFKQMV-EK---------IK--KS---G--AN-VVLCQKG---ID-D--V-AQHYLAKEGIYA-VRRVKKSDMEKLAK----A-T-GAKIV----TDL-D-------D---LTPS-VLGE--A--ETV--E----ERKIGDDRM--TFVMGCK--NPKA-VSI--LI--R---G-GT-DHV------------V----SE--V-ERALND-AIRV----VAITKED--GKFLWGGG-----AVE-----A-----EL---A---M----RLAKYANSVG---G---R-E--QLAI-----E------A------FA---KALEIIPRT-LA-E-----NAGIDPINTLIKLKADD-EKGRISVGVD-LDNNGV----G--D---MKAKGVVDPLRVKTHA--L--E-----S-AV---E-------V--ATMIL-RIDD--VI-
       ||        |       |      |||   |    | |   |   |   |       |    |  | |      |   |    ||    | |     | |   |     |      |   ||   | |    |   |     |   | |   |  | |     ||         |||       |   |      |   |  

found 2649 duplicates! (Removing...)
After removing bad sequences, tpdb is now  80
789
After removing bad sequences, tpdb is now  2

After removing bad sequences...
tpdb (s_ipdb) is :  2
(208, 196)
found bad columns := [ 10  12  18  20  24  29  35  38  39  47  58  59  74  85  91  96 101 116
 117 124 126 130 140 153 155 157 165 167 168 189 192]
found conserved columns (80% repetition):
 [ 35  47  91 189]
We remove conserved and bad columns with, at the following indices (len 31):
 [192 130  10  74  12 140  18  20  85  24 153 155  91 157  29  96  35 165
  38  39 167 168 101  47 116 117  58  59 124 189 126]
Removed Columns...
s now has shape:  (208, 165)
s_index (length=165) = 
 [  0   1   2   3   4   5   6   7   8   9  11  13  14  15  16  17  19  21
  22  23  25  26  27  28  30  31  32  33  34  36  37  40  41  42  43  44
  45  46  48  49  50  51  52  53  54  55  56  57  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  75  76  77  78  79  80  81  82  83  84
  86  87  88  89  90  9